In [3]:
import snappy
from snappy import ProductIO, GPF
import numpy
from osgeo import gdal
from jupytep.common.io import *
from jupytep.maps.leaflet.layer import ImageLayer
from jupytep.maps.mapnik.renders import RenderMap
from jupytep.maps.geoserver.loader import *
from jupytep.maps.content import *

In [4]:
source_path = '/eodata/Sentinel-2/MSI/L1C/2018/05/21/S2B_MSIL1C_20180521T100029_N0206_R122_T33UXT_20180521T120149.SAFE'
source_product = ProductIO.readProduct(source_path)
print(source_product.getProductReader())

org.esa.s2tbx.dataio.s2.l1c.Sentinel2L1CProductReader[input=org.esa.s2tbx.dataio.VirtualPath@7ef41ca2]


In [5]:
target_dir = get_work_dir()
width = source_product.getSceneRasterWidth()
height = source_product.getSceneRasterHeight()

name = source_product.getName()
type = snappy.ProductData.TYPE_INT32
description = source_product.getDescription()
band_names = source_product.getBandNames()

print("Source Product: %s, %d x %d pixels, %s" % (name, width, height, description))
print("Source Bands:   %s" % (list(band_names)))

target = snappy.Product(source_product.getProductType(), source_product.getProductType(), width, height)
snappy.ProductUtils.copyProductNodes(source_product, target)

Source Product: S2B_MSIL1C_20180521T100029_N0206_R122_T33UXT_20180521T120149, 10980 x 10980 pixels, None
Source Bands:   ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'view_zenith_mean', 'view_azimuth_mean', 'sun_zenith', 'sun_azimuth', 'view_zenith_B1', 'view_azimuth_B1', 'view_zenith_B2', 'view_azimuth_B2', 'view_zenith_B3', 'view_azimuth_B3', 'view_zenith_B4', 'view_azimuth_B4', 'view_zenith_B5', 'view_azimuth_B5', 'view_zenith_B6', 'view_azimuth_B6', 'view_zenith_B7', 'view_azimuth_B7', 'view_zenith_B8', 'view_azimuth_B8', 'view_zenith_B8A', 'view_azimuth_B8A', 'view_zenith_B9', 'view_azimuth_B9', 'view_zenith_B10', 'view_azimuth_B10', 'view_zenith_B11', 'view_azimuth_B11', 'view_zenith_B12', 'view_azimuth_B12']


In [6]:
b = source_product.getBand('B4')
print(b)

target_band = snappy.Band('_Enhanced', type, width, height)
print(target_band)
snappy.ProductUtils.copyRasterDataNodeProperties(b, target_band)
target.addBand(target_band)

# target_band = self.target.addBand('Enhanced_VV', snappy.ProductData.TYPE_INT32)

target_band.setNoDataValue(numpy.nan)
target_band.setNoDataValueUsed(True)

r = numpy.zeros(width, numpy.int32)

target_data = get_work_dir() + name+'_Enhanced.dim'
target.setProductWriter(snappy.ProductIO.getProductWriter('BEAM-DIMAP'))
target.writeHeader(target_data)

print("Target Product: %s, %d x %d pixels, %s" % (target.getName(), target.getSceneRasterWidth(), target.getSceneRasterHeight(), target.getDescription()))
print(target.containsBand('_Enhanced'))
target_band.ensureRasterData()

# snappy.ProductUtils.copyGeoCoding(self.source, self.target)
for y in range(height):
    b.readPixels(0, y, width, 1, r)
    target_band.setPixels(0, y, width, 1, r)

target_tif = target_dir + "Snappy_8BitStretch"
# snappy.ProductIO.writeProduct(self.target, self.target_dir+'test.dim', 'BEAM-DIMAP')
snappy.ProductIO.writeProduct(target, target_tif, 'GeoTIFF')
target.closeIO()
target_jpg = target_tif + ".jpg"
target_tif = target_tif + ".tif"
print('Done.')

org.esa.snap.core.datamodel.Band[B4,uint16,10980,10980,3,665.0,90.0,0.0]
org.esa.snap.core.datamodel.Band[_Enhanced,int32,10980,10980,-1,0.0,0.0,0.0]
Target Product: S2B_MSIL1C_20180521T100029_N0206_R122_T33UXT_20180521T120149_Enhanced, 10980 x 10980 pixels, None
True
Done.


In [7]:
load_geotif(target_tif)
show_on_map(target_tif)

http://jupytepide-geoserver:8080/geoserver/rest/imports
CURL: http://jupytepide-geoserver:8080/geoserver/rest/imports/21/tasks
Load tiff id: 0 - DONE
Finish import for ID: 21 
